In [61]:
import pandas as pd
from spectral_cube import SpectralCube
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
from astropy.io import fits

In [62]:
fits_path = ".data/fits/m1_m5.freqtol_1.3MHz.spw014589.clean_line_19.fits"

In [63]:
fits_data = fits.open(fits_path)

In [64]:
# FREQ0 
for el in ["RESTFRQ", "FREQ0", "RESTFREQ"]:
    try:
        print(fits_data[0].header[el])
    except:
        continue

98968514570.0


In [65]:
import numpy as np
import pandas as pd
import math
from spectral_cube import SpectralCube

def fits_to_dataframe(path, noise_frac=0.001):
    # Load the spectral cube
    cube = SpectralCube.read(path)

    df_list = []

    # Iterate over the spectral axis (velocity axis)
    for i in range(cube.shape[0]):
        # Slice one spectral frame at a time
        slab = cube[i, :, :]  # shape: (y, x)

        # Get world coordinates for this frame
        world = slab.wcs.pixel_to_world_values(
            *np.meshgrid(
                np.arange(cube.shape[2]),  # x (RA)
                np.arange(cube.shape[1]),  # y (Dec)
                indexing="xy",
            )
        )

        ra = world[0].flatten()
        dec = world[1].flatten()
        velo = cube.spectral_axis[i].value  # Single velocity value for this slice
        intensity = slab.filled_data[:].value.flatten()

        # For velocity: can do per-pixel (if you want) or per-slice (scalar)
        #velo_noisy = velo + np.random.uniform(-oom_velo, oom_velo)

        # Build DataFrame for this slab
        df_slice = pd.DataFrame(
            {"velocity": velo, "ra": ra, "dec": dec, "intensity": intensity}
        )

        df_list.append(df_slice)

    # Concatenate all slices into a single DataFrame
    df = pd.concat(df_list, ignore_index=True)
    df.dropna(inplace=True)
    
    del cube

    return df


In [66]:
df = fits_to_dataframe(fits_path)
df = df[df["intensity"]>0]

In [73]:
# normalize the df

df_norm = df.apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [74]:
def visualize_with_open3d_filtered(df, intensity_min=None, intensity_max=None, velocity_scale=1):
    # Filter the DataFrame based on intensity
    if intensity_min is not None:
        df = df[df['intensity'] >= intensity_min].copy()  # Create a copy here
    if intensity_max is not None:
        df = df[df['intensity'] <= intensity_max].copy()  # Create a copy here

    # Apply standardized scaling to the velocity
    df['std_Velocity'] = (df['velocity'] - np.mean(df['velocity'])) / np.std(df['velocity'])
    
    # Logarithmic Velocity
    df["log_velocity"] = np.log(df['velocity'] - df['velocity'].min() + 1e-9)

    # Prepare points for visualization (RA, Dec, log-scaled velocity)
    points = np.vstack((-df['ra'], df['dec'], df['log_velocity'] * velocity_scale)).T

    # Create an Open3D point cloud object 
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(points)

    # Normalize intensity for color mapping
    intensity = (df['intensity'] - df['intensity'].mean()) / df['intensity'].std()
    colors = plt.cm.plasma(intensity)[:, :3]

    # Apply colors to the point cloud
    point_cloud.colors = o3d.utility.Vector3dVector(colors)

    # Visualize the point cloud
    o3d.visualization.draw_geometries([point_cloud])

# Example usage
# Assume df is your Pandas DataFrame obtained from the FITS file
# df = fits_to_dataframe('your_file.fits')
visualize_with_open3d_filtered(df_norm, intensity_min=df['intensity'].quantile(0.999), intensity_max=None, velocity_scale=1)

MESA: error: ZINK: failed to choose pdev
glx: failed to create drisw screen
